# 透過自動化的方式爬取人力銀行資料

In [55]:
from selenium import webdriver
from time import sleep
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import requests
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import re

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}
driver_path="/Users/beiyulin/Desktop/chromedriver"
hum_home = "https://www.104.com.tw/jobs/main/"
driver =webdriver.Chrome(executable_path=driver_path)
driver.get(hum_home)
print(driver.current_url)

https://www.104.com.tw/jobs/main/


In [56]:
#sesearch_input = driver.find_element_by_name("#ikeyword")
#search_input.send_keys("資料分析" ,Keys.ENTER)
element =driver.find_element_by_css_selector('#ikeyword')
element.send_keys("資料分析" ,Keys.ENTER )
a=driver.current_url


In [57]:
for i in range(20): 
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    sleep(0.6)



In [58]:
k = 1
while k != 0:
    try:
        # 手動載入新資料之後會出現新的more page，舊的就無法再使用，所以要使用最後一個物件
        driver.find_elements_by_class_name("js-more-page",)[-1].click() 
        # 如果真的找不到，也可以直接找中文!
        driver.find_element_by_xpath("//*[contains(text(),'手動載入')]").click()
        print('Click 手動載入，' + '載入第' + str(15 + k) + '頁')
        k = k+1
        time.sleep(1) # 時間設定太短的話，來不及載入新資料就會跳錯誤
    except:
        k = 0
        print('No more Job')
        


No more Job


In [59]:
element =driver.find_element_by_css_selector('#noticeOnClose') #將跳出的通知按掉
element.click()
sleep(3)

# 以BeautifulSoup進行資料解析

In [60]:

import re,time
joblist=pd.DataFrame()
response = requests.get(a)
soup = BeautifulSoup(response.text, 'html.parser')

#爬取相關的職缺資料

for i in range(0, 20):
    job= soup.find('div',{'id':'js-job-content'}).findAll('article',{'class':'b-block--top-bord job-list-item b-clearfix js-job-item'})[i]
    update_date = job.find('span',{'class':'b-tit__date'}).text
    update_date =re.sub("\n ","",update_date).strip()
    content = job.find('p').text
    content = re.sub("\n  ","",content).strip()
    loc = job.find('ul',{'class':'b-list-inline b-clearfix job-list-intro b-content'}).findAll('li')[0].text
    df=pd.DataFrame(
            data=[{'發布日期' : update_date,
                            '職稱' : job["data-job-name"],
                            '公司名稱' : job["data-cust-name"],
                            '公司類別' : job["data-indcat-desc"],
                            '公司地點' : loc,
                            '工作內容' : content}],
            columns = ['發布日期', '職稱','公司名稱' ,'公司類別','公司地點', '工作內容'])
    joblist= joblist.append(df)
    time.sleep(0.5)
  
    
joblist
#print (joblist)



,發布日期,職稱,公司名稱,公司類別,公司地點,工作內容
0,3/19,資料分析師 (Data Analyst),亞洲指標數位行銷顧問股份有限公司,廣告行銷公關業,台北市松山區,i-Buzz的核心理念是數據推動行銷策略，透過網路語料觀察與混搭資料研究進行開發，提供客戶深...
0,3/11,研究員/博士後研究員(兒少保資料分析),國立台灣大學_中國信託慈善基金會兒少暨家庭研究中心,大專校院教育事業,台北市大安區,1.\t兒少家庭政策及少子化、兒少保護與家暴防治、兒少物質使用及網路成癮議題 (專...
0,3/23,軟體工程師 / AI Engineer / 資料分析師,康信創意科技有限公司,消費性電子產品製造業,台北市內湖區,工作內容\n1. 確認客戶需求，溝通並清楚定義分析問題。\n2. 負責AI軟體之分析、設計以...
0,3/18,Python 資料分析師,干城數碼有限公司,電腦軟體服務業,台北市中山區,1.使用統計、機器學習、深度學習等演算法進行建模\n2.協助定義商業問題以及方法論，定義假說...
0,3/23,Python資料分析師,康翔科技股份有限公司,其它軟體及網路相關業,新北市汐止區,1.使用統計、機器學習、深度學習等演算法進行建模 \n2.協助定義商業問題以及方法論，定義假...
0,3/16,資料分析副理,艾普特媒體有限公司,電腦軟體服務業,台北市中山區,善於溝通、並能理解經營思維\n4. 有管理團隊及專案經驗\n\n擅長工具：\n1. 具統計及...
0,3/17,資訊人員（資料分析人員）,國票綜合證券股份有限公司,證券及期貨業,台北市中山區,1.客戶資料分析\r\n2.客戶行為分析\r\n3.業務分析
0,3/18,Data Analyst 資料分析師,核桃運算股份有限公司,電腦系統整合服務業,台北市大安區,此職務為執行數據服務專案的角色，需具備高度的數據素養，以及相應技術與知識；如果你/妳富有責任...
0,8/18,洗錢防制_數據資料分析人員,中國信託商業銀行股份有限公司,銀行業,台北市南港區,1. AML資料分析\n2. 協助AML系統資料驗證作業\n3. 利用AI及機器學習等技術，...
0,3/18,AI Engineer / 資料分析師,昕傳科技股份有限公司,電腦軟體服務業,台北市松山區,1.負責AI軟體之分析、設計以及程式撰寫。\n2.熟悉至少一種機器學習或資料分析工具或套件：...
